# House Rent Predictor By Sayan

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

In [ ]:
data= pd.read_csv("data\data.csv")

In [ ]:
data

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.hist(bins=50, figsize=(20,15))


In [ ]:
data.columns

In [ ]:
data['CHAS'].value_counts()

In [ ]:
data['CRIM'].value_counts()

In [ ]:
data['RM'].value_counts()

## Train Test Spliting 

In [ ]:
#for learning purpose
#def split_train_test(data, test_ratio):
 #   np.random.seed(42)
 #   shuffled = np.random.permutation(len(data))
  #  test_set_size = int(len(data) * test_ratio)
   # test_indices = shuffled[:test_set_size]
    #train_indices = shuffled[test_set_size:]
    #return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
#train_data, test_data= split_train_test(data, 0.2)

In [ ]:
#print(f"length of train data {len(train_data)} and length test data {len(test_data)}")

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
print(f"length of train data {len(train_data)} and length test data {len(test_data)}")

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for tain_index, test_index in split.split(data, data['CHAS']):
    train_data_set = data.loc[tain_index]
    test_data_set = data.loc[test_index]

In [ ]:
train_data['CHAS'].value_counts()

In [ ]:
train_data_set['CHAS'].value_counts()

In [ ]:
data = train_data_set.copy()

## Looking for correlation

In [ ]:
corr_matrix = data.corr()

In [ ]:
corr_matrix['MEDV'].sort_values(ascending = False)

In [ ]:
from pandas.plotting import scatter_matrix

In [ ]:
attributes = ['RM', 'ZN', 'MEDV', 'LSTAT']
scatter_matrix(data[attributes], figsize=(12, 8))

In [ ]:
data.plot(kind = 'scatter', x= 'RM', y= 'MEDV', alpha = 0.8 )

## Attribute combination

In [ ]:
data['TPR'] = data['TAX'] / data ["RM"]
test_data_set['TPR'] = data['TAX'] / data ["RM"]

In [ ]:
data['TPR']

In [ ]:
data = data.drop("MEDV", axis = 1)

In [ ]:
data_labels = train_data_set["MEDV"].astype(float)
type(data_labels)

## Missing Attribute

In [ ]:
#set the value of RM missing value to value(median)
median = data['RM'].median
median

In [ ]:
data['RM'].fillna(median)

In [ ]:
median1 = data_labels.median()
data_labels.fillna(median1)

In [ ]:
data.describe() #before we strting filling missng attribute

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imp_median =SimpleImputer(strategy='median')
imp_median.fit(data)

In [ ]:
imp_median.statistics_

In [ ]:
x= imp_median.transform(data)
x 

In [ ]:
data_tr = pd.DataFrame(x, columns = data.columns)

In [ ]:
data_tr.describe()

## Scikit -Learn Design

### pipeline create

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
my_pipeline = Pipeline([
    ('Imputer', SimpleImputer(strategy = "median")),
    
    ('std_scaler', StandardScaler())
])

In [ ]:
data_num_tr = my_pipeline.fit_transform(data_tr)

In [ ]:
data_num_tr, data_num_tr.shape

## Selecting a desire model fo perdiction 

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()
model.fit(data_num_tr, data_labels)

In [ ]:
some_data = data_tr.iloc[:5]

In [ ]:
some_labels = data_labels.iloc[:5]

In [ ]:
prepared_data = my_pipeline.transform(some_data)

In [ ]:
model.predict(prepared_data)

In [ ]:
some_labels

In [ ]:
#Error
from sklearn.metrics import mean_squared_error
data_prediction = model.predict(data_num_tr)
lin_mse = mean_squared_error(data_labels, data_prediction)
lin_rmse = np.sqrt(lin_mse)

In [ ]:
lin_rmse

### Decision tree regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model1 = DecisionTreeRegressor()
model1.fit(data_num_tr, data_labels)

In [ ]:
some_data = data_tr.iloc[:5]
some_labels = data_labels.iloc[:5]
prepared_data = my_pipeline.transform(some_data)

In [ ]:
model1.predict(prepared_data)

In [ ]:
some_labels

In [ ]:
#Error
data_prediction = model1.predict(data_num_tr)
lin_mse = mean_squared_error(data_labels, data_prediction)
lin_rmse = np.sqrt(lin_mse)

In [ ]:
lin_rmse

### Random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model2 = RandomForestRegressor()
model2.fit(data_num_tr, data_labels)

In [ ]:
some_data = data_tr.iloc[:5]
some_labels = data_labels.iloc[:5]
prepared_data = my_pipeline.transform(some_data)

In [ ]:
model2.predict(prepared_data)

In [ ]:
some_labels

In [ ]:
#Error
data_prediction = model2.predict(data_num_tr)
lin_mse = mean_squared_error(data_labels, data_prediction)
lin_rmse = np.sqrt(lin_mse)

In [ ]:
lin_rmse

## Using better validation technique - Cross Validation 

In [ ]:
# for Desion Tree
from sklearn.model_selection import cross_val_score
score = cross_val_score(model1, data_num_tr, data_labels, scoring = 'neg_mean_squared_error', cv = 10)
rmse_score_d = np.sqrt(-score)


In [ ]:
rmse_score_d

In [ ]:
# for Regression
score = cross_val_score(model1, data_num_tr, data_labels, scoring = 'neg_mean_squared_error', cv = 10)
rmse_score_r = np.sqrt(-score)

In [ ]:
rmse_score_r

In [ ]:
# for Random forest
score = cross_val_score(model2, data_num_tr, data_labels, scoring = 'neg_mean_squared_error', cv = 10)
rmse_score_ran = np.sqrt(-score)

In [ ]:
rmse_score_ran

In [ ]:
# function for details in scores
def print_scores(score):
    print(f"Scores: {score}")
    print(f"Mean: {score.mean()}")
    print(f"Standard Deviation: {score.std()}")

In [ ]:
print_scores(rmse_score_d)

In [ ]:
print_scores(rmse_score_r)

In [ ]:
print_scores(rmse_score_ran)

## Save model of random forest 

In [ ]:
from joblib import dump, load
dump(model2, 'housing.joblib')

### test model

In [ ]:
x_test = test_data_set.drop('MEDV', axis = 1)
x_test.describe()

In [ ]:
y_test = test_data_set['MEDV'].copy()

In [ ]:
y_test

In [ ]:
x_test_prepared = my_pipeline.transform(x_test)

In [ ]:
final_prediction = model2.predict(x_test_prepared)
final_mse = mean_squared_error(y_test, final_prediction)
final_rmse = np.sqrt(final_mse)

In [ ]:
final_rmse

In [ ]:
# test_data_set.to_csv('test_data.csv', index = False)